In [4]:
%load_ext autoreload
%autoreload 2
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.datasets import make_classification, make_regression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, FeatureUnion
from preprocessor import SavitzkyGolay, Derivate, MultiplicativeScatterCorrection, RobustNormalVariate
import numpy as np
import matplotlib.pyplot as plt

from pipeline_tools import FeatureUnionOnAxis

X, y = make_regression(n_samples=10, n_features=10, random_state=0)
print(X.shape)
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

print(X.shape)
union = FeatureUnionOnAxis([('id', FunctionTransformer()), ('derivate', Derivate()), ('spl', RobustNormalVariate())], axis = 1)
a = union.fit_transform(X, y)
print(a.shape)
# pipe = Pipeline()
# # The pipeline can be used as any other estimator
# # and avoids leaking the test set into the train set
# print(X[0,0:10])
# T = pipe.fit_transform(X_train, y_train)
# print(pipe)
# print(T[0,0:10])


# plt.rcParams['figure.figsize'] = [30,30]
# l = np.arange(0, 1000, 1)
# fig, AX = plt.subplots( 3 )
# for i in range(3):
#     AX[i].plot(l, X[i])
#     AX[i].plot(l, T[i])
    

# pipe.score(X_test, y_test)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
(10, 10)
(10, 10)
(30, 10)


In [8]:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import json
import numpy as np
import sys
import os
import random
sys.path.append(os.path.abspath("") + "/../src/pynirs")
from nirs_set import NIRS_Set as NSet
import preprocessor as pp
import tensorflow as tf
import matplotlib.pyplot as plt
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Conv1D, Flatten, BatchNormalization, SpatialDropout1D, Dropout
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
# from kennard_stone import train_test_split, KFold
# from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from matplotlib.pyplot import plot

SEED = 12485
def set_seed(sd):
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['PYTHONHASHSEED']=str(sd)
    random.seed(sd)
    np.random.seed(sd)
    tf.random.set_seed(sd)
set_seed(SEED)

hello = tf.constant("hello TensorFlow!")

def load_path(folder):
    n = NSet("example")
    n.load(folder+"/XCal.csv", folder+"/YCal.csv", 0, 0, 0)
    x_src = n.get_raw_x()

    print("mean", np.mean(x_src))
    print("std", np.std(x_src))
    print("median", np.median(x_src))
    print("min", np.min(x_src))
    print("max", np.max(x_src))
    print("percentile", np.percentile(x_src,[20, 80]))
    print("linalg", np.linalg.norm(x_src))
    reference = np.mean(x_src, axis = 1)
    print("polyfit", np.polyfit(reference, x_src[:, 0], deg = 1))
    p = np.polyfit(reference, x_src, deg = 1)
    print("polyfit", p)
    print("polyfit", p.shape)
    return
    x = x_src.copy()
    # PROCESSING = ['x','x*detrend','x*snv','x*savgol1','x*msc','x*derivate','x*gaussian1','x*gaussian2','x*wv_haar','x*detrend*snv','x*detrend*rnv','x*detrend*savgol1','x*detrend*msc','x*detrend*derivate','x*detrend*wv_haar','x*wv_bior2.2','x*wv_db2','x*wv_dmey','x*wv_rbio6.8','x*wv_sym4','x*snv*savgol1','x*savgol1*savgol1','x*msc*savgol1','x*derivate*savgol1','x*gaussian1*savgol1','x*gaussian2*savgol1','x*wv_haar*savgol1','x*snv*msc','x*rnv*msc','x*savgol1*msc','x*msc*msc','x*derivate*msc','x*gaussian1*msc','x*gaussian2*msc','x*wv_haar*msc','x*snv*derivate','x*savgol1*derivate','x*msc*derivate','x*derivate*derivate','x*gaussian1*derivate','x*gaussian2*derivate','x*wv_haar*derivate']
    PROCESSING = ['x'
                # ,'x*detrend'
                ,'x*savgol1'
                # ,'x*derivate'
                ,'x*gaussian1'
                ,'x*gaussian2'
                ,'x*wv_haar'
                # ,'x*detrend*savgol1'
                # ,'x*detrend*derivate'
                # ,'x*detrend*wv_haar'
                # # ,'x*detrend*gaussian1'
                # # ,'x*detrend*gaussian2'
                # ,'x*wv_bior2.2'
                # ,'x*wv_db2'
                # ,'x*wv_dmey'
                # ,'x*wv_rbio6.8'
                # ,'x*wv_sym4'
                # ,'x*savgol1*savgol1'
                # ,'x*derivate*savgol1'
                # ,'x*gaussian1*savgol1'
                # ,'x*gaussian2*savgol1'
                # ,'x*wv_haar*savgol1'
                # ,'x*savgol1*derivate'
                # ,'x*derivate*derivate'
                # ,'x*gaussian1*derivate'
                # ,'x*gaussian2*derivate'
                # ,'x*wv_haar*derivate'
                ]
    pp_spectra = pp.process(x, PROCESSING)
    x = np.array(list(pp_spectra.values()))
    x = np.swapaxes(x, 0, 1)
    x = np.swapaxes(x, 1, 2)
    print(x[0,:,0])
    print(x[0,:,1])
    print(x[0,:,2])
    
    x = x_src.copy()
    pp_spectra = pp.process(x[0:2], PROCESSING)
    x = np.array(list(pp_spectra.values()))
    x = np.swapaxes(x, 0, 1)
    x = np.swapaxes(x, 1, 2)
    print(x[0,:,0])
    print(x[0,:,1])
    print(x[0,:,2])
    
    y_src = n.get_raw_y()
    scaler_y = MinMaxScaler(feature_range=(0.1,0.9))
    y = scaler_y.fit_transform(y_src)
    
    return x, y, scaler_y

# def learn(X_train, X_test, y_train, y_test, input_shape, name, folder):
#     model = Sequential()
#     model.add(SpatialDropout1D(0.08, input_shape=input_shape))
#     model.add(Conv1D (filters=8, kernel_size=15, strides=5, activation='selu'))
#     # model.add(BatchNormalization())
#     model.add(Dropout(0.3))
#     model.add(Conv1D (filters=64, kernel_size=21, strides=3, activation='relu'))
#     # model.add(BatchNormalization())
#     model.add(Dropout(0.2))
#     model.add(Conv1D (filters=32, kernel_size=5, strides=3, activation='elu'))
#     # model.add(BatchNormalization())
#     model.add(Dropout(0.2))
#     model.add(Flatten())
#     model.add(Dense(16, activation='sigmoid'))
#     model.add(Dense(1, activation='sigmoid'))

#     # model.summary()

#     reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, verbose=1, min_delta=0.5e-5, mode='min')
#     earlyStopping = EarlyStopping(monitor='val_loss', patience=90, verbose=0, mode='min') 
#     mcp_save = ModelCheckpoint("tmp.model", save_best_only=True, monitor='val_loss', mode='min') 

#     model.compile(loss='mean_squared_error', metrics=['mae','mse'], optimizer='rmsprop')

#     history = model.fit(X_train, y_train, 
#                 epochs=600, 
#                 batch_size=500, 
#                 shuffle=True, 
#                 validation_data = (X_test, y_test),
#                 verbose=0, 
#                 callbacks=[earlyStopping])

#     best_score = min(history.history['val_mse'])
#     best_epoch = np.argmin(history.history['val_mse'])
#     print(folder, input_shape, name, best_score, best_epoch)
#     return best_score
    
    
# def benchmark_folder(x,y,folder):
#     print("***** BENCHMARKING", folder, "*****")
#     s = x.shape
    
#     # X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3)
    
#     scores = {}
    
#     kf = KFold(n_splits=4)#, random_state=SEED, shuffle=True)
#     fold_index = 0
#     # for train_index, test_index in kf.split(x): #sklearn split
#     for train_index, test_index in kf.split(x[:,:,0]): #kennard stone split
        
#         X_train, X_test = x[train_index], x[test_index]
#         y_train, y_test = y[train_index], y[test_index]
        
#         score = learn(X_train, X_test, y_train, y_test, (s[1], s[2]), fold_index, folder)
#         name = str(fold_index)
#         scores[name] = score
            
#         # X_train_src = X_train[:,:,0:2]
#         # X_test_src = X_test[:,:,0:2]
        
#         # for i in range(15, s[2], 15):
#         #     X_train_sub = X_train[:,:,0:i]
#         #     X_test_sub = X_test[:,:,0:i]
#         #     score = learn(X_train_sub, X_test_sub, y_train, y_test, (s[1], i), str(i), folder)
#         #     name = str(fold_index) + '>0-' + str(i)
#         #     scores[name] = score
            
#         # for i in range(15, s[2], 15):
#         #     X_train_sub = X_train[:,:,i-15:i]
#         #     X_train_sub = np.concatenate((X_train_src, X_train_sub), axis = 2)
#         #     X_test_sub = X_test[:,:,i-15:i]
#         #     X_test_sub = np.concatenate((X_test_src, X_test_sub), axis = 2)
#         #     score = learn(X_train_sub, X_test_sub, y_train, y_test, (s[1], 17), str(i), folder)
#         #     name = str(fold_index) + '>' + str(i-15) + "-" + str(i)
#         #     scores[name] = score
            
#         fold_index += 1
        
#     return scores
    
    
def traverse(directory):
    scores = {}
    folders = [x[0] for x in os.walk(directory)]
    for f in folders:
        if f == directory:
            continue
        print("LOADING", f)
        x,y,_ = load_path(f)
        # score = benchmark_folder(x,y,f)
        # scores[f] = score
        break
    return scores

scores = traverse('sample_data')
# with open("scores.json", "w") as write_file:
#     json.dump(scores, write_file, indent=4)

LOADING sample_data\ALPINE_C_424_Murguzur
mean 0.2502768
std 0.22829306
median 0.18485376
min -0.104465626
max 1.3915194
percentile [0.06791676 0.40602568]
linalg 298.51172
polyfit [1.3631628  0.46054249]
polyfit [[1.3631628  1.3960305  1.36012011 ... 1.1916488  1.19263494 1.19377185]
 [0.46054249 0.46635726 0.48146435 ... 0.1880057  0.187743   0.18729798]]
polyfit (2, 2151)


TypeError: cannot unpack non-iterable NoneType object

In [14]:
import numpy as np
X = [[2,2,2,2,2,2,2,2,4],[2,2,2,2,2,2,2,2,0],[2,2,2,2,2,2,2,2,2]]
X = np.array(X)

imax = 1
imin = 0
f = (imax - imin)/(np.max(X) - np.min(X))
n = X.shape
arr = np.empty((0, n[0]), dtype = float) #create empty array for spectra
for i in range(0, n[1]):
    d = X[:, i]
    dnorm = imin + f*d
    arr = np.append(arr, [dnorm], axis = 0)
    
X = np.transpose(arr)
print(X)

n = X.shape
arr = np.empty((0, n[0]), dtype = float) #create empty array for spectra
for i in range(0, n[1]):
    d = X[:, i]
    dnorm = d/f - imin
    arr = np.append(arr, [dnorm], axis = 0)

X = np.transpose(arr)
print(X)

[[0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 1. ]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0. ]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]]
[[2. 2. 2. 2. 2. 2. 2. 2. 4.]
 [2. 2. 2. 2. 2. 2. 2. 2. 0.]
 [2. 2. 2. 2. 2. 2. 2. 2. 2.]]


In [21]:
import numpy as np
X = [[2,2,2,2,2,2,2,2,4],[2,2,2,2,2,2,2,2,0],[2,2,2,2,2,2,2,2,2]]
spectra = X

wave = np.array([1, 2, 1, 2])

p1 = .5 * (wave[0] + wave[-1])
p2 = 2 / (wave[0] - wave[-1])

# Compute model terms
model = np.ones((wave.size(), 4))
model[:, 1] = p2 * (wave[0] - wave) - 1
model[:, 2] = (p2 ** 2) * ((wave - p1) ** 2)
model[:, 3] = np.mean(spectra, axis = 1)

# Solve correction parameters
params = np.linalg.lstsq(model, spectra)[0].T

# Apply correction
spectra = spectra - np.dot(params[:, :-1], model[:, :-1].T).T
spectra = np.multiply(spectra, 1 / np.repeat(params[:, -1].reshape(1, -1), spectra.shape[0], axis = 0))


TypeError: 'int' object is not callable

In [42]:
from scipy import signal
import scipy.ndimage as nd
X = [[2,2,2,3,2,2,2,2,4],[2,2,2,8,2,2,2,2,0],[2,2,2,2,2,2,2,2,2]]
X = np.array(X)

A = np.gradient(X, axis=1)
print(A)
for i in range(len(X)):
    a = A[i]
    x = X[i]
    b = x[0] + 2 * np.c_[np.r_[0, a[1: -1: 2].cumsum()], a[::2].cumsum() - a[0] / 2].ravel()[: len(a)]
    print(b)

[[ 0.   0.   0.5  0.  -0.5  0.   0.   1.   2. ]
 [ 0.   0.   3.   0.  -3.   0.   0.  -1.  -2. ]
 [ 0.   0.   0.   0.   0.   0.   0.   0.   0. ]]
[2. 2. 2. 3. 2. 2. 2. 2. 4.]
[2. 2. 2. 8. 2. 2. 2. 2. 0.]
[2. 2. 2. 2. 2. 2. 2. 2. 2.]


In [11]:
import sys
import os
import random
import preprocessor as pp
from sklearn.preprocessing import StandardScaler

b = pp.StandardNormalVariate()
X = [[1,1,1,0,1,1,1,1,4],[2,2,2,4,2,2,2,2,0],[3,3,3,2,3,3,3,3,2]]
b.fit(X)
# print(b.std_)
x = b.transform(X)
print(x)

print(StandardScaler().fit_transform(X))
# print(b.inverse_transform(x))

[[-1.22474487 -1.22474487 -1.22474487 -1.22474487 -1.22474487 -1.22474487
  -1.22474487 -1.22474487  1.22474487]
 [ 0.          0.          0.          1.22474487  0.          0.
   0.          0.         -1.22474487]
 [ 1.22474487  1.22474487  1.22474487  0.          1.22474487  1.22474487
   1.22474487  1.22474487  0.        ]]
[[-1.22474487 -1.22474487 -1.22474487 -1.22474487 -1.22474487 -1.22474487
  -1.22474487 -1.22474487  1.22474487]
 [ 0.          0.          0.          1.22474487  0.          0.
   0.          0.         -1.22474487]
 [ 1.22474487  1.22474487  1.22474487  0.          1.22474487  1.22474487
   1.22474487  1.22474487  0.        ]]


In [19]:
import numpy as np
from sklearn.preprocessing import *
r = np.random.randn(3,20)
print(r)
t = KernelCenterer().fit_transform(r)
print(t.shape)

[[ 0.20143051  0.1097772  -0.61450787  1.4963892  -0.28725745 -1.51958579
  -0.01017149  0.34425589  0.37918193  1.40525922 -1.22575484 -0.39632028
  -0.17702553  0.6605248   1.39023186  0.04719278  0.3466409  -0.67954721
  -0.67897811  1.32887549]
 [-1.82490933 -0.22867789 -1.81905118  0.23149312  0.03240633  0.73447346
  -0.26554663  0.26920605  0.7099471  -0.80341893  0.46642547  1.2069917
   0.26057525  0.84975242 -0.7164621   0.43009793 -1.46607815 -0.74871409
   0.01645979 -1.20541019]
 [ 0.47786435  0.45886516  1.24086626  0.26425757  0.00950691  1.046114
   0.05481998 -1.45264818  0.76295341 -0.09989035  0.65398573 -0.50767265
  -0.66375955  0.07180835 -1.26407854 -0.11613216  1.37404346 -1.24752758
   0.23331549 -0.5699728 ]]
(3, 80)
